<a href="https://colab.research.google.com/github/rey2512/Model/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import cv2
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from timm import create_model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

BASE_PATH = "/content/drive/MyDrive/1DeepFake/splitted_data"
TRAIN_REAL_PATH = os.path.join(BASE_PATH, "real/train")
TRAIN_FAKE_PATH = os.path.join(BASE_PATH, "fake/train")
VAL_REAL_PATH   = os.path.join(BASE_PATH, "real/val")
VAL_FAKE_PATH   = os.path.join(BASE_PATH, "fake/val")

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

class DeepFakeDataset(Dataset):
    def __init__(self, file_list, labels, transform=None):
        self.file_list = file_list
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        path = self.file_list[idx]
        cap = cv2.VideoCapture(path)
        ret, frame = cap.read()
        cap.release()
        if not ret:
            frame = np.zeros((299, 299, 3), dtype=np.uint8)
        frame = cv2.resize(frame, (299, 299))
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        if self.transform:
            frame = self.transform(frame)
        label = self.labels[idx]
        return frame, label


In [ ]:
import os
import cv2
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.onnx
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import timm
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns

# =======================
# 📌 Define Constants
# =======================
DATASET_PATH = "/content/drive/MyDrive/splitted_data"
MODEL_SAVE_PATH = "/content/drive/MyDrive/best_deepfake_detector2.pth"
BATCH_SIZE = 12  # Increased batch size
EPOCHS = 30
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
PATIENCE = 3  # Early stopping patience

# =======================
# 📌 Data Augmentation & Dataset
# =======================
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

class DeepFakeDataset(Dataset):
    def __init__(self, folder, label, transform=None, frames_per_video=30):
        self.folder = folder
        self.label = label
        self.transform = transform
        self.frames_per_video = frames_per_video
        self.video_files = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith(".mp4")]

    def __len__(self):
        return len(self.video_files)

    def __getitem__(self, idx):
        video_path = self.video_files[idx]
        cap = cv2.VideoCapture(video_path)
        frames = []

        while len(frames) < self.frames_per_video and cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame, (224, 224))
            frame = self.transform(frame)
            frames.append(frame)
        cap.release()

        while len(frames) < self.frames_per_video:
            frames.append(torch.zeros(3, 224, 224))

        return torch.stack(frames), torch.tensor(self.label, dtype=torch.long)

# =======================
# 📌 Load Dataset
# =======================
train_real = DeepFakeDataset(os.path.join(DATASET_PATH, "real/train"), label=0, transform=transform)
train_fake = DeepFakeDataset(os.path.join(DATASET_PATH, "fake/train"), label=1, transform=transform)
val_real = DeepFakeDataset(os.path.join(DATASET_PATH, "real/val"), label=0, transform=transform)
val_fake = DeepFakeDataset(os.path.join(DATASET_PATH, "fake/val"), label=1, transform=transform)

test_real = DeepFakeDataset(os.path.join(DATASET_PATH, "real/test"), label=0, transform=transform)
test_fake = DeepFakeDataset(os.path.join(DATASET_PATH, "fake/test"), label=1, transform=transform)

train_loader = DataLoader(train_real + train_fake, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_real + val_fake, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_real + test_fake, batch_size=BATCH_SIZE, shuffle=False)

# =======================
# 📌 Define Model
# =======================
class DeepFakeDetector(nn.Module):
    def __init__(self):
        super(DeepFakeDetector, self).__init__()
        self.feature_extractor = timm.create_model("efficientnet_lite0", pretrained=True)
        self.feature_extractor.classifier = nn.Identity()
        self.lstm = nn.LSTM(input_size=1280, hidden_size=256, num_layers=1, batch_first=True, dropout=0.3)
        self.fc = nn.Linear(256, 2)

    def forward(self, x):
        B, T, C, H, W = x.shape
        x = x.view(B * T, C, H, W)
        features = self.feature_extractor(x)
        features = features.view(B, T, -1)
        lstm_out, _ = self.lstm(features)
        output = self.fc(lstm_out[:, -1, :])
        return output

# =======================
# 📌 Train Model & Track Performance
# =======================
model = DeepFakeDetector().to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.5)
scaler = torch.cuda.amp.GradScaler()

train_losses, val_losses, val_accuracies = [], [], []
best_val_loss = float('inf')
stopping_rounds = 0

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    train_preds, train_labels = [], [] # Initialize lists to store predictions and labels
    for videos, labels in train_loader:
        videos, labels = videos.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            outputs = model(videos)
            loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()
        train_loss += loss.item()

        # Accumulate predictions and labels for calculating accuracy
        train_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
        train_labels.extend(labels.cpu().numpy())

    train_acc = accuracy_score(train_labels, train_preds) # Calculate training accuracy
    train_losses.append(train_loss / len(train_loader))

    model.eval()
    val_loss, val_preds, val_labels = 0, [], []
    with torch.no_grad():
        for videos, labels in val_loader:
            videos, labels = videos.to(DEVICE), labels.to(DEVICE)
            with torch.cuda.amp.autocast():
                outputs = model(videos)
                loss = criterion(outputs, labels)
            val_loss += loss.item()
            val_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
            val_labels.extend(labels.cpu().numpy())
    val_acc = accuracy_score(val_labels, val_preds)
    val_losses.append(val_loss / len(val_loader))
    val_accuracies.append(val_acc)

    print(f"Epoch [{epoch+1}/{EPOCHS}], Train Loss: {train_loss:.4f}, Train Acc: {train_acc*100:.2f}%, "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc*100:.2f}%")
    scheduler.step()

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
        stopping_rounds = 0
    else:
        stopping_rounds += 1
        if stopping_rounds >= PATIENCE:
            break

# =======================
# 📌 Plot Loss & Accuracy Graphs
# =======================
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss over Epochs')

plt.subplot(1,2,2)
plt.plot(val_accuracies, label='Val Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Validation Accuracy over Epochs')
plt.show()

# =======================
# 📌 Final Test Evaluation
# =======================
model.load_state_dict(torch.load(MODEL_SAVE_PATH))
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for videos, labels in test_loader:
        videos, labels = videos.to(DEVICE), labels.to(DEVICE)
        outputs = model(videos)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
print(classification_report(all_labels, all_preds, digits=4))
cm = confusion_matrix(all_labels, all_preds)

# Plot confusion matrix
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Real', 'Fake'], yticklabels=['Real', 'Fake'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
import torch
import cv2
import timm
import numpy as np
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F

# =======================
# 📌 Define Constants
# =======================
MODEL_PATH = "/content/drive/MyDrive/1DeepFake/best_deepfake_detector2.pth"  # Update path if needed
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
FRAME_SIZE = (224, 224)
FRAMES_PER_VIDEO = 30

# =======================
# 📌 Define Model (Same as Training)
# =======================
class DeepFakeDetector(nn.Module):
    def __init__(self):
        super(DeepFakeDetector, self).__init__()  # Corrected the typo
        self.feature_extractor = timm.create_model("efficientnet_lite0", pretrained=False)
        self.feature_extractor.classifier = nn.Identity()
        # Checking output size of EfficientNet-Lite0
        feature_output_size = self.feature_extractor.num_features
        self.lstm = nn.LSTM(input_size=feature_output_size, hidden_size=256, num_layers=1, batch_first=True, dropout=0.3)
        self.fc = nn.Linear(256, 2)  # Output 2 classes: REAL, FAKE

    def forward(self, x):
        B, T, C, H, W = x.shape
        x = x.view(B * T, C, H, W)  # Flatten time dimension
        features = self.feature_extractor(x)  # Extract features
        features = features.view(B, T, -1)  # Shape (B, T, features)
        lstm_out, _ = self.lstm(features)  # LSTM output
        output = self.fc(lstm_out[:, -1, :])  # Take the last time step
        return output

# Load model
model = DeepFakeDetector().to(DEVICE)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.eval()

# =======================
# 📌 Preprocessing Function
# =======================
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(FRAME_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

def preprocess_video(video_path, frames_per_video=FRAMES_PER_VIDEO):
    cap = cv2.VideoCapture(video_path)
    frames = []

    while len(frames) < frames_per_video and cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = transform(frame)  # Apply transformations
        frames.append(frame)
    cap.release()

    while len(frames) < frames_per_video:
        frames.append(torch.zeros(3, *FRAME_SIZE))  # Padding if not enough frames

    return torch.stack(frames).unsqueeze(0)  # Shape: (1, T, C, H, W)

# =======================
# 📌 Inference Function
# =======================
def predict(video_path):
    video_tensor = preprocess_video(video_path).to(DEVICE)  # Preprocess video

    with torch.no_grad():
        output = model(video_tensor)  # Get model output
        probabilities = F.softmax(output, dim=1)  # Convert logits to probabilities
        fake_prob = probabilities[0, 1].item() * 100  # Get probability of fake class

    label = "FAKE" if fake_prob > 50 else "REAL"  # Classify as fake or real
    print(f"Prediction: {label} | Fake Probability: {fake_prob:.2f}%")

# =======================
# 📌 Run Inference
# =======================
video_path = "/content/drive/MyDrive/Ncsc /WhatsApp Video 2025-04-04 at 22.24.05_f291f0c6.mp4"  # Replace with your test video path
predict(video_path)


Prediction: FAKE | Fake Probability: 51.73%
